In [ ]:
!cd "/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis"
!pip install transformers


     |████████████████████████████████| 2.1MB 6.6MB/s 
     |████████████████████████████████| 901kB 22.8MB/s 
     |████████████████████████████████| 3.3MB 35.3MB/s 


In [ ]:
import torch
import pandas as pd
import numpy as np

# Thu vien transformer cho Classification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification

# Xu ly label
from sklearn.preprocessing import LabelEncoder

# Metric danh gia 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

# Ve do thi
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
# Xay dung data de fit vao mo hinh 
class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Doc du lieu
!cd "/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis"
train = pd.read_excel("/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis/dataset/train_nor_811.xlsx", index_col=False)
dev = pd.read_excel("/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis/dataset/valid_nor_811.xlsx", index_col=False)
test = pd.read_excel("/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis/dataset/test_nor_811.xlsx", index_col=False)

X_train = train['Sentence']
y_train = train['Emotion'].values

X_dev = dev['Sentence']
y_dev = dev['Emotion'].values

X_test = test['Sentence']
y_test = test['Emotion'].values

print(len(train))
print(len(dev))
print(len(test))

5548
686
693


In [ ]:
import torch
from transformers import BertTokenizer,BertModel
from transformers import AutoModel, AutoTokenizer

# Define the model name in transformers repository
model = AutoModelForSequenceClassification.from_pretrained("NlpHUST/vibert4news-base-cased", num_labels = 7)
tokenizer = BertTokenizer.from_pretrained("NlpHUST/vibert4news-base-cased")

Some weights of the model checkpoint at NlpHUST/vibert4news-base-cased were not used when initializing IBertForSequenceClassification: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.wei

In [ ]:
# Chuan bi data
le = LabelEncoder()
le.fit(y_train)
# encoding du lieu
train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True, max_length=40)
dev_encodings = tokenizer(X_dev.to_list(), truncation=True, padding=True, max_length=40)
test_encodings = tokenizer(X_test.to_list(), truncation=True, padding=True, max_length=40)

y_train_encoding = le.transform(y_train)
y_dev_encoding = le.transform(y_dev)
y_test_encoding = le.transform(y_test)

In [ ]:
# Chuan bi data

train_dataset = BuildDataset(train_encodings, y_train_encoding)
dev_dataset = BuildDataset(dev_encodings, y_dev_encoding)
test_dataset = BuildDataset(test_encodings, y_test_encoding)

In [ ]:
# Chuan bi mo hinh

training_args = TrainingArguments(
    output_dir='./results_xlmr',          
    num_train_epochs=5,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,   
    warmup_steps=500,                
    weight_decay=0.01,
    no_cuda=False
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=dev_dataset             
)

In [ ]:
trainer.train()

Step,Training Loss
500,1.630400


TrainOutput(global_step=870, training_loss=1.3775563207165948, metrics={'train_runtime': 763.6383, 'train_samples_per_second': 1.139, 'total_flos': 889872971275200.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 971083776, 'init_mem_gpu_alloc_delta': 1068132864, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 12505088, 'train_mem_gpu_alloc_delta': 1616796160, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 1022579712})

In [ ]:
# Du doan 
y_pred_classify = trainer.predict(test_dataset)

y_pred_xlmr = np.argmax(y_pred_classify.predictions, axis=-1)
np.save("/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis/y_pred_news.npy",y_pred_xlmr)

In [13]:
# Danh gia mo hinh
y_true = y_test_encoding
np.save("/content/drive/MyDrive/NLP-DS/Thực hành 1 - Sentiment analysis/y_true.npy",y_true)
cf = confusion_matrix(y_true, y_pred_xlmr)
print(cf)

evaluation = f1_score(y_true, y_pred_xlmr, average='micro')

print("F1 - micro: " + str(evaluation))

evaluation = f1_score(y_true, y_pred_xlmr, average='macro')
print("F1 - macro: " + str(evaluation))

[[  8  19   1   1   6   5   0]
 [  6  68  30   6  16   5   1]
 [  1  14 134   5  25  12   2]
 [  0   6   6  27   2   5   0]
 [  1  17  26   1  67  13   4]
 [  1  10  28   4  13  60   0]
 [  0   4  11   1  11   1   9]]
F1 - micro: 0.5382395382395382
F1 - macro: 0.48475428190855857


In [ ]:
# Ve ma tran nham lan
df_cm = pd.DataFrame(cf, index = np.unique(y_train),
                  columns = np.unique(y_train))

sn.heatmap(df_cm, annot=True, cmap="Greys",fmt='g', cbar=True, annot_kws={"size": 10})